# This is to demo GenAI prompt in FSI

## Financial Analysis Generation by Template

Sell side analyst needs to write analysis report regularly, and there are some template that they use. following is a sample report on Apple Q4 2020 by a CFA in MorningStar about its quarterly financial report. 


![CFA report on Apple](https://d1i7pm9zcth9lt.cloudfront.net/applecfa.png)

It mentions:
* FVE
* Revenue analysis by product sectors or region
* Gross margin
* Analyst view or anticipation

After Apple release a new quarterly report Q1 2021, the CFA would need to write a new analysis report. 

In [1]:
from IPython.display import HTML

pdf_path = 'https://d1i7pm9zcth9lt.cloudfront.net/FY21+Q1+Consolidated+Financial+Statements.pdf'
html = f'<iframe src="{pdf_path}" width="100%" height="600px"></iframe>'
HTML(html)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/IPython/core/display.py:431: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


GenAI might be able to help with prompt below.

In [1]:
prompt1 = """
You are a CFA analyst, and following is an analysis report of 30 Oct 2020. You can use it as a template.

<template>
Apple’s Mac and iPad Sales Remain Bolstered by Work-From-Home Trend in Q4; Raising FVE to $85

Apple reported fiscal fourth-quarter results ahead of our expectations led by Mac and iPad segments.
The firm did not provide guidance the last two quarters and again refrained from offering specific guidance due to uncertainty regarding COVID-19. CEO Tim Cook expects iPhone revenue to grow in the December quarter despite the new iPhone 12 being launched a couple of weeks later in the quarter, though he did not specify the magnitude of growth. Meanwhile, all other products and services are expected to grow in the double digits.
We are raising our fair value estimate for narrow-moat Apple to $85 per share from $71 as we incorporate a stronger near-term outlook for the Mac and iPad segments due to ongoing work- and learning-from-home dynamics. Nonetheless, we think shares are currently overvalued, as we think recent growth trends could be unsustainable as we enter 2021.
Fourth-quarter revenue was up 1% year over year thanks to growth in iPad (46%), Mac (29%), services (16%), and wearables, home, and accessories (21%). Management noted the iPad and Mac segments remained supply constrained, which bodes well for these business lines in the December quarter.
Apple’s iPhone sales were understandably down 21% year over year due to the iPhone 12 delay. Apple now has over 585 million paid subscribers to its various services, up 135 million from a year ago, and the firm expects 600 million subs by Dec 2020. Although Greater China was the region most impacted by the absence of the new iPhones for the quarter (total revenue down 29% year over year), non-iPhone sales grew double digits. Gross margin of 38.2% was up 20 basis points sequentially due to a higher mix of services.
Management was optimistic revenue from Greater China would grow in the first quarter, particularly as 5G is more mature in the region. We anticipate iPhone revenue for fiscal 2021 will be up in the low teens, though the late launch may shift some sales from the December quarter to the March quarter.
</template>

Now, apple releases the latest Apple Reports First Quarter Results below.
<finance>
Apple Inc.
CONDENSED CONSOLIDATED STATEMENTS OF OPERATIONS (Unaudited)
(In millions, except number of shares which are reflected in thousands and per share amounts)
Three Months Ended
December 26,
2020
December 28,
2019
Net sales:
Products $ 95,678 $ 79,104
Services 15,761 12,715
Total net sales (1) 111,439 91,819
Cost of sales:
Products 62,130 52,075
Services 4,981 4,527
Total cost of sales 67,111 56,602
Gross margin 44,328 35,217
Operating expenses:
Research and development 5,163 4,451
Selling, general and administrative 5,631 5,197
Total operating expenses 10,794 9,648
Operating income 33,534 25,569
Other income/(expense), net 45 349
Income before provision for income taxes 33,579 25,918
Provision for income taxes 4,824 3,682
Net income $ 28,755 $ 22,236
Earnings per share:
Basic $ 1.70 $ 1.26
Diluted $ 1.68 $ 1.25
Shares used in computing earnings per share:
Basic 16,935,119 17,660,160
Diluted 17,113,688 17,818,417
(1) Net sales by reportable segment:
Americas $ 46,310 $ 41,367
Europe 27,306 23,273
Greater China 21,313 13,578
Japan 8,285 6,223
Rest of Asia Pacific 8,225 7,378
Total net sales $ 111,439 $ 91,819
(1) Net sales by category:
iPhone $ 65,597 $ 55,957
Mac 8,675 7,160
iPad 8,435 5,977
Wearables, Home and Accessories 12,971 10,010
Services 15,761 12,715
Total net sales $ 111,439 $ 91,819
Apple Inc.
CONDENSED CONSOLIDATED BALANCE SHEETS (Unaudited)
(In millions, except number of shares which are reflected in thousands and par value)
December 26,
2020
September 26,
2020
ASSETS:
Current assets:
Cash and cash equivalents $ 36,010 $ 38,016
Marketable securities 40,816 52,927
Accounts receivable, net 27,101 16,120
Inventories 4,973 4,061
Vendor non-trade receivables 31,519 21,325
Other current assets 13,687 11,264
Total current assets 154,106 143,713
Non-current assets:
Marketable securities 118,745 100,887
Property, plant and equipment, net 37,933 36,766
Other non-current assets 43,270 42,522
Total non-current assets 199,948 180,175
Total assets $ 354,054 $ 323,888
LIABILITIES AND SHAREHOLDERS’ EQUITY:
Current liabilities:
Accounts payable $ 63,846 $ 42,296
Other current liabilities 48,504 42,684
Deferred revenue 7,395 6,643
Commercial paper 5,000 4,996
Term debt 7,762 8,773
Total current liabilities 132,507 105,392
Non-current liabilities:
Term debt 99,281 98,667
Other non-current liabilities 56,042 54,490
Total non-current liabilities 155,323 153,157
Total liabilities 287,830 258,549
Commitments and contingencies
Shareholders’ equity:
Common stock and additional paid-in capital, $0.00001 par value: 50,400,000 shares
authorized; 16,823,263 and 16,976,763 shares issued and outstanding, respectively 51,744 50,779
Retained earnings 14,301 14,966
Accumulated other comprehensive income/(loss) 179 (406)
Total shareholders’ equity 66,224 65,339
Total liabilities and shareholders’ equity $ 354,054 $ 323,888
Apple Inc.
CONDENSED CONSOLIDATED STATEMENTS OF CASH FLOWS (Unaudited)
(In millions)
Three Months Ended
December 26,
2020
December 28,
2019
Cash, cash equivalents and restricted cash, beginning balances $ 39,789 $ 50,224
Operating activities:
Net income 28,755 22,236
Adjustments to reconcile net income to cash generated by operating activities:
Depreciation and amortization 2,666 2,816
Share-based compensation expense 2,020 1,710
Deferred income tax benefit (58) (349)
Other 25 (142)
Changes in operating assets and liabilities:
Accounts receivable, net (10,945) 2,015
Inventories (950) (28)
Vendor non-trade receivables (10,194) 3,902
Other current and non-current assets (3,526) (7,054)
Accounts payable 21,670 (1,089)
Deferred revenue 1,341 985
Other current and non-current liabilities 7,959 5,514
Cash generated by operating activities 38,763 30,516
Investing activities:
Purchases of marketable securities (39,800) (37,416)
Proceeds from maturities of marketable securities 25,177 19,740
Proceeds from sales of marketable securities 9,344 7,280
Payments for acquisition of property, plant and equipment (3,500) (2,107)
Payments made in connection with business acquisitions, net (9) (958)
Other 204 (207)
Cash used in investing activities (8,584) (13,668)
Financing activities:
Proceeds from issuance of common stock — 2
Payments for taxes related to net share settlement of equity awards (2,861) (1,379)
Payments for dividends and dividend equivalents (3,613) (3,539)
Repurchases of common stock (24,775) (20,706)
Proceeds from issuance of term debt, net — 2,210
Repayments of term debt (1,000) (1,000)
Proceeds from/(Repayments of) commercial paper, net 22 (979)
Other (22) (16)
Cash used in financing activities (32,249) (25,407)
Decrease in cash, cash equivalents and restricted cash (2,070) (8,559)
Cash, cash equivalents and restricted cash, ending balances $ 37,719 $ 41,665
Supplemental cash flow disclosure:
Cash paid for income taxes, net $ 1,787 $ 4,393
Cash paid for interest $ 619 $ 771
</finance>

As a CFA analyst, please give me an updated analysis report following the template above.
"""

In [14]:
import boto3
import json

session = boto3.Session(region_name='us-west-2') 
bedrock = session.client(service_name="bedrock-runtime")

messages = [{"role": "user", "content": prompt1}]
body = {"messages": messages, 
        "max_tokens": 10000, 
        "temperature": 0.5, 
        "anthropic_version":"", 
        "top_k": 250, 
        "top_p": 1, 
        "stop_sequences": ["Human"]}
response = bedrock.invoke_model(modelId="anthropic.claude-3-sonnet-20240229-v1:0", body=json.dumps(body))
text_resp = json.loads(response['body'].read().decode('utf-8'))
print(text_resp['content'][0]['text'])



Here is an updated analysis report on Apple's fiscal Q1 2021 results following the template:

Apple's iPhone Sales Rebound in Q1 as Work-From-Home Trend Continues; Maintaining FVE at $85 

Apple reported fiscal first-quarter results ahead of our expectations driven by a strong rebound in iPhone sales and continued momentum in services. 

Revenue of $111.4 billion was up 21% year-over-year, with double-digit growth across all product categories and services. iPhone revenue of $65.6 billion increased 17% despite the delayed launch timing, benefiting from strong demand for the first 5G-enabled iPhone 12 lineup. Services revenue grew 24% to $15.8 billion, with paid subscriptions surpassing 620 million. 

We are maintaining our $85 fair value estimate for narrow-moat Apple as we incorporate the impressive first-quarter results and outlook. However, shares appear modestly overvalued as we believe some of the recent elevated growth trends may prove unsustainable as we move through 2021.

iPho

You can find the generated report includes following information as well:
* FVE
* Revenue analysis by product sectors or region
* Gross margin
* Analyst view or anticipation

Cloud3 correctly understand and calculate the right YOY, such as the Revnew  21% YOY, iPhone 17% YOY, etc.

You always needs to verify the output to avoid hullucination. 

If you are not satisfactory, you can run several times to get multiple generated reports to get inspirations. Following is another sample generated report comparing with the original report by human CFA.

![Comparison of CFA report on Apple](https://d1i7pm9zcth9lt.cloudfront.net/CFAreport2.jpg)